**Описание проекта**

Обучение модели линейной регрессии на данных о жилье в Калифорнии в 1990 году.

На основе данных нужно предсказать медианную стоимость дома в жилом массиве — median_house_value.

Для оценки качества модели используются метрики RMSE, MAE и R2.

# Импортирование библиотек и загрузка данных

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a7a32d77215d0d30320ca74bad670fdfc59af245337cb0f0e1a6c28a88245cea
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler

RANDOM_SEED = 987

In [3]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California house median price prediction") \
                    .getOrCreate()

**Описание данных**

В колонках датасета содержатся следующие данные:

longitude — широта;

latitude — долгота;

housing_median_age — медианный возраст жителей жилого массива;

total_rooms — общее количество комнат в домах жилого массива;

total_bedrooms — общее количество спален в домах жилого массива;

population — количество человек, которые проживают в жилом массиве;

households — количество домовладений в жилом массиве;

median_income — медианный доход жителей жилого массива;

median_house_value — медианная стоимость дома в жилом массиве;

ocean_proximity — близость к океану.

In [4]:
df_housing = spark.read.load('/content/drive/MyDrive/Colab Notebooks/California Median House Price Prediction/housing.csv', format="csv", sep=",", inferSchema = True, header=True)
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
pd.DataFrame(df_housing.dtypes, columns=['column', 'type'])

,column,type
0,longitude,double
1,latitude,double
2,housing_median_age,double
3,total_rooms,double
4,total_bedrooms,double
5,population,double
6,households,double
7,median_income,double
8,median_house_value,double
9,ocean_proximity,string


Все столбцы за исключением одного, который имеет строковый тип данных, имеют тип числа с плавающей точкой

In [6]:
df_housing.limit(5).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [7]:
print('shape: ', (df_housing.count(), len(df_housing.columns)))

shape:  (20640, 10)


Данные имеют 20640 строк и 10 фичей

# Подготовка данных

Проверим данные на наличие пропусков

In [8]:
df = df_housing.select([F.count(F.when(F.col(c).contains('None') | \
                            F.col(c).contains('NULL') | \
                            (F.col(c) == '' ) | \
                           F.col(c).isNull() | \
                            F.isnan(c), c
                           )).alias(c)
                    for c in df_housing.columns])
df.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,0,0,0,207,0,0,0,0,0


В столбце total_bedrooms содержится 207 пропусков. Избавимся от них.

In [9]:
df_housing = df_housing.dropna()

Разделим фичи на категориальные, количественные и таргет

In [10]:
cat_cols = ['ocean_proximity']
num_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
            'total_bedrooms', 'total_bedrooms', 'population', 'households', 'median_income']
target = 'median_house_value'

Трансфорируем категориальный признак и применим технику OHE.

In [11]:
indexer = StringIndexer(inputCols=cat_cols,
                        outputCols=[cat_cols[0] +'_idx'])
df_housing = indexer.fit(df_housing).transform(df_housing)

In [12]:
encoder = OneHotEncoder(inputCols=[c + '_idx' for c in cat_cols], outputCols=[c + '_ohe' for c in cat_cols])
df_housing = encoder.fit(df_housing).transform(df_housing)

In [13]:
cat_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in cat_cols], outputCol="cat_features")
df_housing = cat_assembler.transform(df_housing)

In [14]:
num_assembler = VectorAssembler(inputCols=num_cols, outputCol="num_features")
df_housing = num_assembler.transform(df_housing)

Применим масштабирование к численным столбцам.

In [15]:
standardScaler = StandardScaler(inputCol='num_features', outputCol="num_features_scaled")
df_housing = standardScaler.fit(df_housing).transform(df_housing)

In [16]:
df_housing.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'ocean_proximity_idx',
 'ocean_proximity_ohe',
 'cat_features',
 'num_features',
 'num_features_scaled']

In [17]:
num_and_cut_assembler = VectorAssembler(inputCols= ['cat_features','num_features_scaled'],
                                  outputCol="features")
num_assembler = VectorAssembler(inputCols= ['num_features_scaled'],
                                  outputCol="features")
df_housing_nc = num_and_cut_assembler.transform(df_housing)
df_housing_n = num_assembler.transform(df_housing)

In [18]:
df_housing_nc.select(['cat_features','num_features_scaled']).show(3)

+-------------+--------------------+
| cat_features| num_features_scaled|
+-------------+--------------------+
|(4,[3],[1.0])|[-61.005863841998...|
|(4,[3],[1.0])|[-61.000872770752...|
|(4,[3],[1.0])|[-61.010854913244...|
+-------------+--------------------+
only showing top 3 rows



У нас есть два датафрейма: df_housing_n - только с количественными столбцами и df_housing_nc - который содержит и количественные, и категориальные столбцы.

# Обучение моделей

Для оценкир моделей будем использовать три метрики: RMSE, MAE и R2.

Модель, использующая только количественные столбцы

In [19]:
train, test = df_housing_n.randomSplit([.8, .2], seed=RANDOM_SEED)
lr = LinearRegression(labelCol=target, featuresCol='features', regParam=0.3)
model = lr.fit(train)
predictions = model.transform(test)
trainingSummary = model.summary
print('Model with only num features:')
print("RMSE: %0.2f" % trainingSummary.rootMeanSquaredError)
print('MAE: %0.2f' % trainingSummary.meanAbsoluteError)
print("r2: %0.2f" % trainingSummary.r2)

Model with only num features:
RMSE: 69488.94
MAE: 50889.12
r2: 0.64


Модель, использующая как столбцы с количественными, так и с категориальными данными

In [20]:
train, test = df_housing_nc.randomSplit([.8, .2], seed=RANDOM_SEED)
lr = LinearRegression(labelCol=target, featuresCol='features', regParam=0.3)
model = lr.fit(train)
predictions = model.transform(test)
trainingSummary = model.summary
print('Model with num and cat features:')
print("RMSE: %0.2f" % trainingSummary.rootMeanSquaredError)
print('MAE: %0.2f' % trainingSummary.meanAbsoluteError)
print("r2: %0.2f" % trainingSummary.r2)

Model with num and cat features:
RMSE: 68586.65
MAE: 49878.90
r2: 0.65


По всем трем метрикам модель, содержащая и количественные, и категориальные столбцы, оказалась немного лучше.

Завершим spark-сессию.

In [21]:
spark.stop()

**Вывод**

**В результате работы были подготовлены данные и обучены две модели линейной регрессии для предсказания медианной стоимости дома в жилом массиве Калифорнии в 1990 году с использованием библиотеки Spark. Результаты показывают, что модель, обученная как на количественных, так и на категориальных столбцах показала лучшие метрики: RMSE и MAE у нее меньше, а R2 - больше, чем у той модели, которая была обучена только лишь на количественных столбцах.**